In [179]:
import pandas as pd
from textblob import TextBlob
import mord
from sklearn import linear_model, metrics, preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
import numpy as np

In [158]:
rev = pd.read_csv("cinema_review.csv")
Y_star = rev['stars']
rev_text = pd.Series(rev['text'])

In [18]:
rev_depo = list()
for r in rev_text:
    rev_tb = TextBlob(r)
    reviews = ' '.join(rev_tb.words.singularize())
    rev_depo.append(reviews)

In [151]:
attributes = open("nouns.txt").read().split()
designX = pd.DataFrame(columns=attributes)
for attr in attributes:
    tb=list()
    for r in rev_depo:
        temp = r.split()
        indices = [i for i,x in enumerate(temp) if x == attr]
        
        if len(indices)==0: 
            tb.append(0)
            continue
        
        temp_sum=0    
        for index in indices:
            if index < 5:
                temp_sum+=TextBlob(' '.join(temp[0:index+6])).sentiment[0]
            elif index > len(temp)-5:
                temp_sum+=TextBlob(' '.join(temp[index-6:])).sentiment[0]
            else:
                temp_sum+=TextBlob(' '.join(temp[index-6:index+6])).sentiment[0]
        temp_avg=temp_sum/len(indices)
        tb.append(temp_avg)
    designX[attr]=pd.Series(tb)

In [168]:
le = preprocessing.LabelEncoder()
le.fit(Y_star)
Y_star = le.transform(Y_star)

In [193]:
clf = mord.LogisticAT(alpha=100)
clf.fit(designX, Y_star)
print(clf.classes_)
print(clf.coef_)
print('Mean Absolute Error of LogisticAT %s' %
      metrics.mean_absolute_error(clf.predict(designX), Y_star))

[0 1 2 3 4]
[0.940692   0.98898316 1.01521799 0.41129533 1.27842165 0.99500993
 0.68370721 0.77701897 0.56525693 0.41805724 1.15998343 0.51321814
 0.60904784 1.06503371 0.46132126 0.6257373  0.27032258 0.38899353
 0.25523397 0.33513774 0.46637685 0.27923213 0.01918263 0.18197612
 0.45591644 0.49790936 0.74631098 0.1085484  0.34768925 0.10539861
 0.18283149 0.70064603 0.60659473]
Mean Absolute Error of LogisticAT 0.8906492097923768
